In [2]:
import os
import json
import numpy as np
import cv2
from PIL import Image

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:


def create_mask_from_polygon(polygons, image_size):
    mask = np.zeros(image_size, dtype=np.uint8)
    for polygon in polygons:
        points = np.array([polygon], dtype=np.int32)
        cv2.fillPoly(mask, points, color=(255))
    return mask





In [4]:
def vgg_json_to_masks(json_file, image_dir, output_dir):
    with open(json_file) as f:
        annotations = json.load(f)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename, annotation in annotations.items():
        image_path = os.path.join(image_dir, filename)
        image = Image.open(image_path)
        image_size = (image.height, image.width)  # Use the image dimensions

        polygons = []
        for region in annotation['regions'].values():
            if region['shape_attributes']['name'] == 'polygon':
                all_points_x = region['shape_attributes']['all_points_x']
                all_points_y = region['shape_attributes']['all_points_y']
                polygon = list(zip(all_points_x, all_points_y))
                polygons.append(polygon)

        mask = create_mask_from_polygon(polygons, image_size)

        mask_filename = os.path.splitext(filename)[0] + '_mask.png'
        mask_path = os.path.join(output_dir, mask_filename)
        cv2.imwrite(mask_path, mask)


In [5]:

json_file_path = '/content/drive/MyDrive/Colab Notebooks/cell_segmentation_and_classifier/Segmentation_data_and_labels/Segmentation_labels.json'
image_dir_path = '/content/drive/MyDrive/Colab Notebooks/cell_segmentation_and_classifier/Segmentation_data_and_labels/Segmentation_Data'
output_dir_path = '/content/drive/MyDrive/Colab Notebooks/cell_segmentation_and_classifier/Segmentation_data_and_labels/Masks'


In [6]:
vgg_json_to_masks(json_file_path, image_dir_path, output_dir_path)
